# Combined Datasets

In [ ]:
# Importing dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Data

Data explanation placeholder

(Talk about original two notebooks?)

In [ ]:
# Reading in data
df_movies = pd.read_csv("./Resources/movies_data.csv")
df_economics = pd.read_csv("./Resources/economics_data.csv")

### Movie Data

### Economics Data

# Combining Data

In [ ]:
# Creating a 'Date' for a datetime index
df_movies['Date'] = pd.to_datetime({
    'year': df_movies['released_year'],
    'month': df_movies['released_month'],
    'day': df_movies['released_day']
})

# Setting `Date` as index
df_movies.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_movies.sort_index(inplace=True)

In [ ]:
# Creating a `Year` and `Month` for concatenation
df_economics['Year'] = df_economics['Date'].str.slice(0,4).astype(int)
df_economics['Month'] = df_economics['Date'].str.slice(5,7).astype(int)

# Renaming to `Year` and `Month` for concatenation
df_movies.rename(columns={
'released_year': 'Year',
'released_month': 'Month'
}, inplace=True)

In [ ]:
# Confirming total records before concatenation
print(f'Total ecomonic records: {df_economics.shape[0]}')
print(f'Total movie records: {df_movies.shape[0]}')

In [ ]:
# Combining datasets through concatenation
df_combined = pd.merge(df_economics, df_movies, how='left', on=['Year', 'Month'])

# Confirming total records after concatenation
print(f'Total records: {df_combined.shape[0]}')

# EDA

In [ ]:
# Creating the eventual `Target` for modelling
df_combined['Target'] = df_combined['critical_success'] + ' ' +\
                        df_combined['financial_success'] + ' ' +\
                        df_combined['Economic Climate']

In [ ]:
# Creating a list of features to drop
cols_to_drop = [
    'Date',
    'CCI Rolling Mean',
    'CPI Rolling Mean',
    'Unemployment Rate (%) Rolling Mean',
    'Economic Climate',
    'Year',
    'Month',
    'id',
    'cast',
    'original_language',
    'director',
    'writers',
    'producers',
    'popularity', 
    'critical_success',
    'financial_success',
    'release_date',
    'released_day',
    'production_countries',
    'status',
    'spoken_languages'
]

# Dropping unneeded features
df_combined.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# Dropping `NaN` records
df_combined.dropna(inplace=True)

In [ ]:
# Confirming total records after concatenation
print(f'Total records: {df_combined.shape[0]}')

# Train Test Splitting

In [ ]:
# Defining columns to scale and encode
col_to_scale = [
    'CCI Value', 'CCI Rolling Percent Change', 'CPI Value',
    'CPI Rolling Percent Change', 'Unemployment Rate (%)', 
    'Unemployment Rate Rolling Percent Change','vote_average', 'vote_count',
    'revenue','runtime','budget', 'roi'
]

col_to_encode = [
    'CCI Rolling Percent Change Flag', 'CPI Rolling Percent Change Flag',
    'Unemployment Rate Rolling Percent Change Flag', 'title', 'original_title',
    'genres', 'production_companies'
]

# Setup X and y variables
X = df_combined.drop(columns='Target')
y = df_combined['Target']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

# Scaling and Econding

In [ ]:
# Creating an instance for `StandardScalar()`
scaler = StandardScaler()

# Fitting and transforming to `col_to_scale`
X_train_scaled = scaler.fit_transform(X_train[col_to_scale])
X_test_scaled = scaler.transform(X_test[col_to_scale])

# Converting results to DF for later concatenation
X_train_scaled = pd.DataFrame(X_train_scaled, columns=col_to_scale)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=col_to_scale)

In [ ]:
# Creating an instance for `OneHotEncoder()` for `X_train[col_to_encode]`
encoder_x = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fitting to `col_to_encode`
encoder_x.fit(X_train[col_to_encode])

# Creating an instance for `OneHotEncoder()` for `y_train`
encoder_y = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

#Fitting
encoder_y.fit(y_train.values.reshape(-1,1))

# Transforming `X_train[col_to_encode]` and `X_test[col_to_encode]`
X_train_encoded = encoder_x.transform(X_train[col_to_encode])
X_test_encoded = encoder_x.transform(X_test[col_to_encode])

# Transforming `y_train` and `y_test`
y_train_encoded = encoder_y.transform(y_train.values.reshape(-1,1))
y_test_encoded = encoder_y.transform(y_test.values.reshape(-1,1))

# Converting results to DF for later concatenation
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder_x.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder_x.get_feature_names_out())

In [ ]:
# Concatenating the `col_to_scale` with `col_to_encode` for `X_train` and `X_test`
X_train = pd.concat([X_train_scaled, X_train_encoded], axis=1)
X_test = pd.concat([X_test_scaled, X_test_encoded], axis=1)

In [ ]:
# Confirming total records after concatenation
print(f'Total X records: {X_train.shape[0] + X_test.shape[0]}')

# Modeling

Playtime!!

# Eric's Space

# Funda's Space

# Kalvin's Space

# Odele's Space

# Peta's Space

# Vadim's Space

# Findings

# Additional